In [1]:
import pandas as pd
import numpy as np
import datetime
import gc
from detect_sleep_states.import_data import import_data
from sklearn.cluster import KMeans
from xgboost import XGBClassifier, XGBRegressor
from tqdm import tqdm  
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
merged_df, mappings = import_data("data/train_series.parquet","data/train_events.csv" )

/home/blinor/.cache/pypoetry/virtualenvs/detect-sleep-states-HW9MEIZd-py3.11/lib/python3.11/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
100%|██████████| 4127301/4127301 [00:44<00:00, 91930.96it/s]
/home/blinor/.cache/pypoetry/virtualenvs/detect-sleep-states-HW9MEIZd-py3.11/lib/python3.11/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
100%|██████████| 309/309 [00:00<00:00, 88352.30it/s]]

100%|██████████| 310/310 [00:00<00:00, 173921.11it/s]


In [3]:
merged_df.info(), merged_df.memory_usage(deep=True)/1024/1024

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127946340 entries, 0 to 127946339
Data columns (total 11 columns):
 #   Column     Dtype  
---  ------     -----  
 0   series_id  uint16 
 1   step       uint32 
 2   anglez     float32
 3   enmo       float32
 4   seconds    uint32 
 5   day        uint8  
 6   month      uint8  
 7   year       uint8  
 8   weekday    uint8  
 9   event      int8   
 10  night      int16  
dtypes: float32(2), int16(1), int8(1), uint16(1), uint32(2), uint8(4)
memory usage: 3.0 GB


(None,
 Index          0.000122
 series_id    244.038277
 step         488.076553
 anglez       488.076553
 enmo         488.076553
 seconds      488.076553
 day          122.019138
 month        122.019138
 year         122.019138
 weekday      122.019138
 event        122.019138
 night        244.038277
 dtype: float64)

In [20]:
train_x = []
train_y = []
limit = 0
for group, data in tqdm(merged_df.dropna().groupby("series_id")):
    train_x.extend(
        [window.values for window in data[["enmo","anglez"]].rolling(window=721, center=True) if len(window.values)==721])
    train_y.extend(
        [window.values[360] for window in data["event"].rolling(window=721, center=True) if len(window.values)==721])
    if limit >5:
        break
    else:
        limit+=1
        

  2%|▏         | 6/277 [01:14<56:00, 12.40s/it]


In [21]:
xgbr = XGBRegressor(tree_method="gpu_hist", device="gpu")
xgbr.fit(train_x, train_y)

ValueError: Please reshape the input data into 2-dimensional matrix.

In [ ]:
plt.plot(xgbr.predict(train_x[:37200]))
plt.plot(train_y[:37200])
plt.plot(merged_df.iloc[:37200]["enmo"])

In [ ]:
tmp = merged_df.query("series_id=='fe90110788d2'").copy()
# tmp["timestamp"] = pd.to_datetime(tmp["timestamp"])

In [ ]:
tmp["moving_enmo"] = tmp["enmo"].rolling(360, center=False).mean()
tmp["var_enmo"] = tmp["enmo"].rolling(360, center=False).var()
tmp=tmp.dropna()

In [ ]:
kmeans = KMeans(n_clusters=2, random_state=0, n_init="auto").fit(tmp[["moving_enmo"]])
tmp["pred"] =kmeans.labels_

In [ ]:
tmp = tmp.replace({"wakeup":1, "onset":0})
tmp["error"] = (tmp["event"]-tmp["pred"]).abs()
tmp["error"].sum()/tmp.shape[0]
f1_score(tmp["event"], tmp["pred"])


In [ ]:
tmp.query("timestamp <'2017-08-10'").set_index("timestamp")[[ "enmo","moving_enmo","var_enmo","event"]].plot(alpha=0.9, secondary_y=["event"], figsize=(25,5))

In [ ]:
tmp.query("event==0").moving_enmo.plot.density()
tmp.query("event==1").moving_enmo.plot.density()
plt.xlim(0, 0.5)

In [ ]:
tmp.query("event==0").enmo.plot.density()
tmp.query("event==1").enmo.plot.density()
plt.xlim(0, 0.5)

In [ ]:
tmp["mult"] = tmp["enmo"]*tmp["anglez"]

In [ ]:
tmp.query("event==0").anglez.plot.density()
tmp.query("event==1").anglez.plot.density()
plt.legend()

In [ ]:
tmp.query("event==0").mult.plot.density()
tmp.query("event==1").mult.plot.density()
plt.xlim(-10,10)